In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
#import chart_studio.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
import torch

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning:

The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



In [2]:
from keras.callbacks import ModelCheckpoint

In [3]:
# Train Data
df_train_en = pd.read_csv('../input/tamilmisodatanew/ta-en-misogyny-train.csv', sep='\t', header=None)
df_train_ta = pd.read_csv('../input/tamilmisodatanew/ta-misogyny-train.csv', sep='\t', header=None)

# Dev Data
df_dev_en = pd.read_csv('../input/tamilmisodatanew/ta-en-misogyny-dev.csv', sep='\t', header=None)
df_dev_ta = pd.read_csv('../input/tamilmisodatanew/ta-misogyny-dev.csv', sep='\t', header=None)

# Test Data
df_test_en = pd.read_csv('../input/tamilmisodatanew/ta-en-misogyny-test.csv', sep='\t', header=None)
df_test_ta = pd.read_csv('../input/tamilmisodatanew/ta-misogyny-test.csv', sep='\t', header=None)

In [4]:
df_train_ea = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_A_train.csv', sep = '\t', header = None)
df_dev_ea = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_A_dev.csv', sep = '\t', header = None)
df_test_ea = pd.read_csv('../input/emotion-analysis-in-tamil/Emotion_Analysis_Task_A_test.csv', sep = '\t', header = None)


In [5]:
train_data = df_train_en.dropna()
val_data = df_dev_en.dropna()
test_data = df_test_en.dropna()


In [6]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 64000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 128
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(train_data[1])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 24541 unique tokens.


In [7]:
X_train = tokenizer.texts_to_sequences(train_data[1])
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_train.shape)

X_val = tokenizer.texts_to_sequences(val_data[1])
X_val = pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_val.shape)

X_test = tokenizer.texts_to_sequences(test_data[1])
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Shape of data tensor: (5943, 128)
Shape of data tensor: (1486, 128)
Shape of data tensor: (1857, 128)


In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8539879927576009440
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15667560448
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14010426360812890186
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


2022-03-07 05:04:04.775047: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-07 05:04:04.829505: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-07 05:04:04.830686: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-07 05:04:04.831395: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [9]:
Y_train = pd.get_dummies(train_data[0]).values
print('Shape of label tensor:', Y_train.shape)

Y_val = pd.get_dummies(val_data[0]).values
print('Shape of label tensor:', Y_val.shape)

Shape of label tensor: (5943, 8)
Shape of label tensor: (1486, 8)


In [10]:
def get_predicted(preds):
    pred_flat = np.argmax(preds, axis=1).flatten()
    return pred_flat
from sklearn.metrics import f1_score, accuracy_score
# def f1(ytrue, ypred):
#     return f1_score(get_predicted(np.array(ytrue)), get_predicted(np.array(ypred)), average = 'weighted')

In [11]:
from keras import backend as K
def f1(true, pred): #shapes (batch, 4)

    #for metrics include these two lines, for loss, don't include them
    #these are meant to round 'pred' to exactly zeros and ones
    #predLabels = K.argmax(pred, axis=-1)
    #pred = K.one_hot(predLabels, 4) 


    ground_positives = K.sum(true, axis=0)       # = TP + FN
    pred_positives = K.sum(pred, axis=0)         # = TP + FP
    true_positives = K.sum(true * pred, axis=0)  # = TP
        #all with shape (4,)

    precision = (true_positives + K.epsilon()) / (pred_positives + K.epsilon()) 
    recall = (true_positives + K.epsilon()) / (ground_positives + K.epsilon()) 
        #both = 1 if ground_positives == 0 or pred_positives == 0
        #shape (4,)

    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
        #not sure if this last epsilon is necessary
        #matematically not, but maybe to avoid computational instability
        #still with shape (4,)

    weighted_f1 = f1 * ground_positives / K.sum(ground_positives)
    weighted_f1 = K.sum(weighted_f1)


    return weighted_f1 #for metrics, return only 'weighted_f1

In [12]:
mc = ModelCheckpoint('model_tam_1.h5', monitor='val_f1', mode='max', save_best_only=True)
cb_list = mc

In [13]:
from tensorflow.keras import layers

In [14]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(layers.Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1])
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=4, batch_size=32, callbacks = cb_list)
# Final evaluation of the model
model_pred_train = model.predict(X_train)
model_pred_test = model.predict(X_val)
# print(classification_report(test_labels,model_pred_test))
#print('LSTM Recurrent Neural Network baseline: ' + str(roc_auc_score(Y_train, model_pred_train)))
#print('LSTM Recurrent Neural Network: ' + str(roc_auc_score(Y_val, model_pred_test)))

2022-03-07 05:04:09.915609: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-07 05:04:09.916464: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-07 05:04:09.917099: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-07 05:04:09.917985: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-07 05:04:09.918632: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Epoch 1/4
186/186 [==============================] - 208s 1s/step - loss: 1.2275 - f1: 0.4690 - val_loss: 1.1128 - val_f1: 0.5375
Epoch 2/4
186/186 [==============================] - 205s 1s/step - loss: 0.8086 - f1: 0.6452 - val_loss: 1.0837 - val_f1: 0.6205
Epoch 3/4
186/186 [==============================] - 206s 1s/step - loss: 0.4808 - f1: 0.7735 - val_loss: 1.2173 - val_f1: 0.6386
Epoch 4/4
186/186 [==============================] - 205s 1s/step - loss: 0.2945 - f1: 0.8506 - val_loss: 1.3664 - val_f1: 0.6399


In [15]:

model_pred_train = model.predict(X_train)
model_pred_test = model.predict(X_val)
f1 = f1_score(get_predicted(Y_train), get_predicted(model_pred_train), average = 'weighted')
acc = accuracy_score(get_predicted(Y_train), get_predicted(model_pred_train))
print(f1, acc)

0.9092949151101495 0.9187279151943463


In [16]:
f1 = f1_score(get_predicted(Y_val), get_predicted(model_pred_test), average = 'macro')
f1

0.3468107999516442

In [17]:
acc = accuracy_score(get_predicted(Y_val), get_predicted(model_pred_test))
print(f1, acc)

0.3468107999516442 0.6951547779273217


In [18]:
for x in get_predicted(Y_train): print(x)

5
6
5
5
5
6
5
5
5
0
3
5
1
5
5
3
5
5
0
0
5
5
5
3
5
5
5
5
5
5
0
5
5
5
5
5
3
5
5
5
5
5
5
5
5
3
5
6
5
3
5
5
5
5
2
5
1
5
5
5
5
5
3
3
7
5
5
5
5
5
5
5
5
3
5
5
1
3
7
5
5
5
5
5
5
5
5
4
5
5
5
5
5
5
5
7
4
6
5
5
5
3
6
5
5
5
3
5
5
4
5
5
3
5
3
7
5
7
5
1
7
5
5
0
3
0
5
5
5
4
5
5
4
5
6
5
3
5
3
7
1
3
5
5
3
5
5
5
3
5
5
5
3
1
5
5
5
5
5
6
5
5
7
5
5
5
5
3
2
7
5
0
5
5
0
5
3
5
5
5
5
5
3
5
4
5
5
5
5
3
3
5
5
4
5
5
5
0
5
5
5
7
0
5
3
5
5
5
5
3
5
5
5
5
3
5
5
5
5
5
5
3
3
5
0
5
7
3
5
2
5
5
5
3
5
5
5
4
5
3
5
3
5
5
5
5
5
5
5
5
3
3
5
3
4
5
1
5
4
3
5
5
3
5
5
5
3
5
5
5
2
5
5
0
5
5
5
3
7
5
1
5
5
5
5
5
4
5
5
5
5
5
5
5
5
0
7
4
5
5
4
5
5
5
5
5
5
3
5
4
4
5
5
5
5
7
5
5
5
0
5
5
5
5
6
5
1
5
5
5
5
5
5
5
5
3
5
0
4
5
5
5
7
5
2
3
5
5
5
5
5
5
5
5
5
6
6
7
7
0
5
5
5
5
3
5
7
5
1
5
0
5
5
3
0
3
5
5
3
3
5
1
3
5
5
5
0
5
5
5
5
5
5
5
1
5
6
5
5
5
5
1
5
2
1
3
3
5
6
5
3
5
5
5
5
5
5
3
5
3
5
3
5
5
7
5
3
7
5
5
5
0
2
5
0
5
5
7
2
5
1
5
5
5
4
5
7
5
5
2
5
6
5
1
5
2
3
1
1
5
3
5
2
5
5
5
3
0
5
5
5
5
5
5
3
5
5
5
3
5
5
0
5
1
5
7
5
4
5
5
7
5
4
5
5
5
0
5
5
5


In [19]:
from sklearn.metrics import classification_report
report = classification_report(get_predicted(Y_val), get_predicted(model_pred_test),output_dict=True)


In [20]:
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
0,0.552632,0.221053,0.315789,95.000000
1,0.187500,0.069767,0.101695,43.000000
2,0.250000,0.075472,0.115942,53.000000
3,0.549801,0.633028,0.588486,218.000000
4,0.391304,0.180000,0.246575,50.000000
5,0.768733,0.906216,0.831832,917.000000
6,0.057143,0.050000,0.053333,40.000000
7,0.961538,0.357143,0.520833,70.000000
accuracy,0.695155,0.695155,0.695155,0.695155
macro avg,0.464831,0.311585,0.346811,1486.000000
